# Coverage and power of simple experiments

Jelle, October 2023

In [ ]:
import jax
import nafi
import nafi.high_level as nhl
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# For precise CLS p-values
jax.config.update("jax_enable_x64", True)

# Save all plots in this directory
plot_dir = Path('plots_cov_and_power')
plot_dir.mkdir(exist_ok=True)

def save_plot(name):
    plt.savefig(plot_dir / f"{name}.png", dpi=300, bbox_inches='tight')
    plt.show()

First, consider a Gaussian measurement $x \sim \mathrm{Normal}(\mu, 1)$, with $\mu \geq 0$. 

This is the high-background limit of a counting experiment, i.e. constraining a signal $\mu_s$ in the presence of a background $\mu_b \rightarrow \infty$ (and $\mu_b \gg \mu_s$) with the count $n \sim \mathrm{Poisson}(\mu_s + \mu_b) \rightarrow \mathrm{Normal}(\mu_s + \mu_b, \sqrt{\mu_b})$ or equivalently $x \equiv \frac{n - \mu_b}{\sqrt{\mu_b}} \sim \mathrm{Normal}(\mu_s, 1)$.

In [ ]:
# Only consider a finite range of mu_sig and x
# Results will be unreliable for extreme x and high mu!
x_range = (-10, 12)
mu_sig = 12 * np.linspace(0, 1, 1_000)**2
lnl, weights, x = nafi.likelihoods.gaussian.lnl_and_weights(
    mu_sig,
    n_x =10_000,
    x_transform=lambda p: x_range[0] + (x_range[1] - x_range[0]) * p,
    return_outcomes=True
)

gaussian = nhl.Experiment(
    name=r"High-background counting experiment",
    cl=0.9,
    lnl=lnl,
    weights=weights,
    hypotheses=mu_sig,
    outcomes=x,
    #outcome_label="Observed x / $\sigma$",
    outcome_label = "Observed $(n - \mu_{\mathrm{bg}}) / \sqrt{\mu_\mathrm{bg}}$",
    #hypothesis_label="Signal $\mu \;/\; \sigma$",
    hypothesis_label="$\mu_\mathrm{sig} \;/\; \\sqrt{\mu_\mathrm{bg}}$",
    outcome_plot_range=(-5, 5),
    hypothesis_plot_range=(0, 4),
    #singular_is_empty=True
)

Next, consider a more sophisticated experiment that discriminates signal from background along some observable (say energy). Specifically, suppose the energy spectra of signal and background are equal-width Gaussians with a separation of three times their width.

The second experiment has a background of 100 events. (The inifinite-background case would reduce back to the previous experiment.)

In [ ]:
mu_bg = 100
sigma_sep = 3

# Signal rate hypotheses to consider: include 0 exactly, almost zero,
# then 200 values at linearly increasing separation.
res = 1.2
mu_sig = np.cumsum(np.concatenate([
    [0, .00001],
    np.linspace(0.05/res, 0.3/res, int(100*res))]))
n_max_sig = nafi.large_n_for_mu(mu_sig.max())

lnl, weights = nafi.likelihoods.two_gaussians.lnl_and_weights(
    mu_sig,
    mu_bg=mu_bg,
    n_sig_max=n_max_sig,
    # TODO: 10k would be nicer...
    trials_per_n=5_000,
    sigma_sep=sigma_sep)

gauss_sep = nhl.Experiment(
    name=f'Two Gauss, {mu_bg}-event bg. at {sigma_sep}$\sigma$ separation',
    lnl=lnl,
    weights=weights,
    hypotheses=mu_sig,
    hypothesis_label="Signal events $\mu$",
    hypothesis_plot_range=(0, 6),
    #singular_is_empty=True,
)

In [ ]:
experiments = dict(gaussian=gaussian, gauss_sep=gauss_sep)

# Hack to avoid the region where CLs gives numerical errors.
# Not needed for the plots below, and may require singular_is_empty=True above.
# gaussian.get_results(nhl.CLs)
# q = gaussian._results[nhl.CLs].p_outcome
# q['mistake_ul'] = np.where(q['mistake_ul'] < 1e-5, np.nan, q['mistake_ul'])

## One-sided methods

The plots below shows the size $\alpha$ and rejection power $\mathrm{RP}$ for simple, classic 90% confidence limit, upper limits.

In [ ]:
for xp_name, xp in experiments.items():
    method_styles = {
        nhl.ClassicUL: dict(color='g'),
    }
    nhl.power_vs_mistake(
        xp, method_styles=method_styles, logit=False,
        which='ul',
        subplots=False, title=True,
        linewidth=1, legend_kwargs=dict(frameon=True))
    nhl.plots.brazil_hspan(yscale=100)
    save_plot(f'{xp_name}_ul_alpha_rp_classiconly')

Clearly for some hypotheses $\alpha \approx \mathrm{RP}$, i.e. exclusions are almost as likely when the hypothesis is true as when the dominant alternative ($\mu = 0$) is true instead. Such exclusions would not be very informative.

PCL and CLs both aim to avoid these cases. PCL maintains $\alpha$ at nominal (10% for a 90% C.L.) unless $\mathrm{RP}$ drops below some threshold (e.g. 50% for median-PCL), and if so, refuses to exclude any lower hypotheses. CLs instead requires $\alpha/\mathrm{RP} > \alpha_\mathrm{nominal}$, i.e. shrinks $\alpha$ to $\alpha = \alpha_\mathrm{nominal} \cdot \mathrm{RP}$.

For the high-background case, either median-PCL or CLs may have a lower $\alpha$, depending on the hypothesis. In the example with discrimination, CLs always has the lower $\alpha$. Since they are still based on the same test statistic / ordering of observations, CLs limits in the latter case are strictly higher than the median-PCL limits.

In [ ]:
for xp_name, xp in experiments.items():

    method_styles = {
        nhl.ClassicUL: dict(color='g'),
        nhl.ULPCLMedian: dict(color='b'),
        nhl.CLs: dict(color='r'),
        #nhl.BayesUL: dict(color='orange'),
    }

    nhl.power_vs_mistake(
        xp, method_styles=method_styles, logit=False,
        which='ul',
        subplots=False, title=True,
        linewidth=1, legend_kwargs=dict(frameon=True))
    nhl.plots.brazil_hspan(yscale=100)
    #plt.axhline(50, c='k', lw=1, alpha=0.2)
    save_plot(f'{xp_name}_ul_alpha_rp')

## Two-sided methods

If we want our methods to also give lower limits on appropriate results -- in case we are interested in actually discovering something ;-) -- there is a broader choice of methods.

We have two basic methods:

  * **Central intervals**: probability of excluding the truth is equally high for both limits (e.g. 5% for a 90% confidence interval);
  * **Unified / Feldman-Cousins**: for each hypothesis, include outcomes in order of the likelihood ratio (the test statistic).

These coincide only for trivial cases such as a free Gaussian measurement (with $\mu \in \mathbb{R}$, not the Gaussian measurement with $\mu \geq 0$ we're considering). Otherwise, unified intervals have unequal error probabilities, i.e. the truth may be more likely to be above than below the interval. Likewise, central intervals do not order by the likelihood ratio, so there exist a pair of outcomes where only the _less_ extreme one (as judged by the test statistic) leads to a rejection of a hypothesis.

Do not confuse central intervals with symmetric confidence intervals, i.e. where the upper and lower limit are equidistant from the best-fit. Those again arise only in special cases.

The plot below shows the Neyman bands for these two methods, for the Gaussian experiment. For the discrimination experiment, we cannot sort and enumerate outcomes in an easy way to make this plot. 

In [ ]:
gaussian.hypothesis_plot_range= (0, 6)
gauss_sep.hypothesis_plot_range = (0, 10)

In [ ]:
method_styles = method_styles_base = {
    nhl.FeldmanCousins: dict(color='steelblue'),
    nhl.CentralIntervals: dict(color='seagreen'),
}

nhl.coverage_credibility_plot(gaussian, method_styles, lw=1.5)
cov_ax = plt.gcf().get_axes()[-1]
ax = cov_ax.twiny()
ax.set_xlim(cov_ax.get_xlim())
ax.set_xticks(cov_ax.get_xticks())
ax.set_xticklabels(["{:.0f}".format((100 - p)) for p in cov_ax.get_xticks()])
ax.set_xlabel(r"$\alpha$, type I error [%]")
ax.xaxis.set_ticks_position('top')

save_plot('twosided_bands')

Both methods can result in either an upper limit, or both an upper and a lower limit. Central intervals can also result in an empty result. Feldman-Cousins intervals never become empty since there is always some hypothesis for which the likelihood ratio is maximal. However, Feldman-Cousins upper limits can become very low (arbitrarily low?).

PCL can be easily applied to two-sided methods: we simply constrain the upper limit based on the rejection power of the upper limit. For example, for a 90% central confidence interval, the upper limit is a 95% classic upper limit ($\alpha = 5\%$), for which [CCGV](https://arxiv.org/abs/1105.3166) recommended a $-1 \sigma$ (16%) power constraint. 

Some experiments also prefer a **discovery threshold**: a requirement that an upper limit is only present once the p-value of the background-only hypothesis is below some threshold; we consider $3 \sigma$ here. You can also consider combinations of PCL and a discovery threshold.

The plots below show the coverage of our two different experiments, for no PCL, then $-1\sigma$ PCL, and finally median-PCL.

In [ ]:
def twosided_outcome_plot(
        xp, method_styles, outcome='mistake',
        ylabel=r"$\alpha$, size, [%]",
        ylabel_complement="C.L., [%]",
        ylim=(0, 15),
        yticks=(0, 5, 10, 15),
        reference_line=10,
        complement_axis=True,
    ):

    nrows = len(method_styles)
    _, axes = plt.subplots(nrows=nrows, ncols=1, figsize=(3, 0.5 + nrows), sharex=True)

    for (ax_i, ax), (method, style) in zip(enumerate(axes), method_styles.items()):
        r = xp.get_results(method)
        plt.sca(ax)

        if outcome == 'discovery':
            p_ul = r.p_outcome['degenerate']
            p_both = p_ul + r.p_outcome['discovery']
        else:
            p_both = r.p_outcome[outcome]
            p_ul = r.p_outcome[outcome + '_ul']

        plt.plot(xp.hypotheses, 100 * p_both, color=style['color'], lw=1)
        plt.fill_between(xp.hypotheses, 0, 100 * p_ul, color=style['color'], lw=0, alpha=0.2)

        if reference_line is not None:
            plt.axhline(reference_line, color='k', lw=1, alpha=0.2)

        show_ylabel = ax_i in (0, nrows - 1)

        plt.xlim(xp.hypothesis_plot_range)
        if show_ylabel:
            plt.ylabel(ylabel)
        ax.set_ylim(*ylim)
        if yticks is None:
            yticks = ax.get_yticks()
        yticks = [q for q in yticks if ylim[0] <= q <= ylim[1]]
        yticklabels = (
            [yticks[0] if ax_i == nrows - 1 else ""]
            + yticks[1:-1]
            + [yticks[-1] if ax_i == 0 else f"{yticks[0]}\n{yticks[-1]}"])
        ax.set_yticks(yticks, yticklabels, fontsize=8)

        # Complement axis
        if complement_axis:
            ax2 = plt.twinx()
            ax2.set_ylim(ax.get_ylim())
            if show_ylabel:
                ax2.set_ylabel(ylabel_complement, alpha=0.5)
            ax2.set_yticks(yticks)
            # Label yticks with complements
            yticks_c = (100 - np.asarray(yticks)).tolist()
            ax2.set_yticklabels(
                [yticks_c[0] if ax_i == nrows - 1 else f"{yticks_c[0]}\n{yticks_c[-1]}"]
                + yticks_c[1:-1]
                + [yticks_c[-1] if ax_i == 0 else ""],
                fontsize=8, alpha=0.5)

        # Text on upper left corner
        plt.text(0.05, 0.9, method.name, transform=ax.transAxes, fontsize=8, va='top', ha='left',
                bbox=dict(facecolor='w', alpha=0.6, lw=0)
                )
        # Text in bottom right corner
        plt.text(0.95, 0.07, "Upper limit alone", transform=ax.transAxes, fontsize=8, va='bottom', ha='right',
                color=style['color'], alpha=0.5)

    plt.sca(axes[-1])
    plt.xlabel(xp.hypothesis_label)
    plt.subplots_adjust(hspace=0)

method_styles_vanilla = method_styles_base | {
    nhl.FCThreeSigmaDisc: dict(color='goldenrod', linestyle='--', zorder=5),
    nhl.CentralIntervalsThreeSigmaDisc: dict(color='orchid', linestyle=':', zorder=5, lw=2),
    #nhl.CLsFlipFlopThreeSigma: dict(color='firebrick'),
}

method_styles_pclm1s = {
    nhl.FeldmanCousinsPCLMinusOne: dict(color='steelblue'),
    nhl.CentralIntervalsPCLMinusOne: dict(color='seagreen'),
    nhl.FCThreeSigmaDiscPCLMinusOne: dict(color='goldenrod', linestyle='--', zorder=5),
    nhl.CIThreeSigmaDiscPCLMinusOne: dict(color='orchid', linestyle=':', zorder=5, lw=2),
}

method_styles_pclmed = {
    nhl.FeldmanCousinsPCLMedian: dict(color='steelblue'),
    nhl.CentralIntervalsPCLMedian: dict(color='seagreen'),
    nhl.FCThreeSigmaDiscPCLMedian: dict(color='goldenrod', linestyle='--', zorder=5),
    nhl.CIThreeSigmaDiscPCLMedian: dict(color='orchid', linestyle=':', zorder=5, lw=2),
}

method_sets = [
    ('nopcl', method_styles_vanilla),
    ('pclm1s', method_styles_pclm1s),
    ('pclmed', method_styles_pclmed),
]

for method_set_name, method_styles in method_sets:
    print(method_set_name)
    for xp_name, xp in experiments.items():
        twosided_outcome_plot(xp, method_styles)
        #plt.suptitle(xp.name, y=0.95, fontsize=10)
        save_plot(f"coverage_both_{xp_name}_{method_set_name}")

For Feldman-Cousins, PCL introduces funny curves in the coverage profile. For central intervals, PCL adds a simple step to the coverage -- it appears to be a steep diagonal in the discrimination experiment only because of the limited resolution of hypotheses we probed.

A discovery threshold has the simpler effect of letting the size start near zero and then monotonously increasing towards the nominal value. (Coverage starts near 100% then decreases towards the nominal value.) Combining discovery threshold and PCL combines both effects.

CLs is harder to generalize to two-sided methods. We consider two ways below:

  * Naive flip-flop: publish a Feldman-Cousins or Central Intervals interval when a discovery threshold is met, otherwise a one-sided CLs upper limit. As with the flip-flop method originally considered in [Feldman & Cousins](https://arxiv.org/abs/physics/9711021), this has undercoverage for a large range of hypotheses and seems untenable.
  * Clipped intervals: publish a Feldman-Cousins or Central Intervals interval, but raise the upper limit to the CLs upper limit if the latter is higher. This produces strictly conservative results (without undercoverage). A discovery threshold is optional (whereas it was mandatory for the flip-flop method).

In [ ]:
class CLsFlipFlopCIThreeSigma(nhl.FlipFlopMethod):
    min_sigma = 3
    method_if_discovery = nhl.CentralIntervals
    method_otherwise = nhl.CLs

method_styles = {
    nhl.CLs: dict(color='r'),
    nhl.CLsFlipFlopThreeSigma: dict(color='firebrick'),
    CLsFlipFlopCIThreeSigma: dict(color='darksalmon'),
    nhl.FeldmanCousinsCLsClipped: dict(color='peru'),
    nhl.FCThreeSigmaCLsClipped: dict(color='saddlebrown'),
    nhl.CICLsClipped: dict(color='maroon'),
    nhl.CIThreeSigmaCLsClipped: dict(color='sienna'),
}

for xp_name, xp in experiments.items():
    twosided_outcome_plot(xp, method_styles)
    save_plot(f"coverage_both_{xp_name}_cls_variants")

As with PCl, the coverage profiles can be pretty funky. Central intervals clipped to CLs looks the simplest -- indeed, that looks like a smooth version of using central intervals with PCL at the median.

Similarly, we can plot the rejection power for different methods:

In [ ]:
gaussian.hypothesis_plot_range = (0, 4)
gauss_sep.hypothesis_plot_range = (0, 6)

for method_set_name, method_styles in method_sets:
    print(method_set_name)
    for xp_name, xp in experiments.items():
        twosided_outcome_plot(xp, method_styles, 'excl_if_bg',
                              ylabel='Rej.power [%]',
                              ylim=(0, 60), yticks=(0, 20, 40, 60),
                              reference_line=None, complement_axis=False)
        save_plot(f"rp_both_{xp_name}_{method_set_name}")

The rejection power of the upper limit is still monotonously increasing, as you would expect. The total rejection power of both limits is __not__ monotonous, unless a discovery treshold essentially eliminates the rejection power of the lower limit.